# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fdd94b59b20>
├── 'a' --> tensor([[-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398]])
└── 'x' --> <FastTreeValue 0x7fdd94b59c10>
    └── 'c' --> tensor([[ 1.0154,  1.2901,  0.0250,  0.2991],
                        [ 0.8841, -0.1253, -0.4294, -0.0852],
                        [ 0.7056,  0.7251,  1.1095, -0.7267]])

In [4]:
t.a

tensor([[-0.0328, -1.7972, -0.0108],
        [ 0.4859, -0.6701, -0.2398]])

In [5]:
%timeit t.a

63.3 ns ± 0.0256 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fdd94b59b20>
├── 'a' --> tensor([[ 0.9550, -0.7758,  0.4918],
│                   [ 0.5984, -0.3373, -0.1643]])
└── 'x' --> <FastTreeValue 0x7fdd94b59c10>
    └── 'c' --> tensor([[ 1.0154,  1.2901,  0.0250,  0.2991],
                        [ 0.8841, -0.1253, -0.4294, -0.0852],
                        [ 0.7056,  0.7251,  1.1095, -0.7267]])

In [7]:
%timeit t.a = new_value

63.4 ns ± 0.0654 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398]]),
    x: Batch(
           c: tensor([[ 1.0154,  1.2901,  0.0250,  0.2991],
                      [ 0.8841, -0.1253, -0.4294, -0.0852],
                      [ 0.7056,  0.7251,  1.1095, -0.7267]]),
       ),
)

In [10]:
b.a

tensor([[-0.0328, -1.7972, -0.0108],
        [ 0.4859, -0.6701, -0.2398]])

In [11]:
%timeit b.a

59.4 ns ± 0.0164 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6411,  2.1806,  0.6083],
               [ 0.4028, -0.2716, -1.4892]]),
    x: Batch(
           c: tensor([[ 1.0154,  1.2901,  0.0250,  0.2991],
                      [ 0.8841, -0.1253, -0.4294, -0.0852],
                      [ 0.7056,  0.7251,  1.1095, -0.7267]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 23.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdce77ca460>
├── 'a' --> tensor([[[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]],
│           
│                   [[-0.0328, -1.7972, -0.0108],
│                    [ 0.4859, -0.6701, -0.2398]]])
└── 'x' --> <FastTreeValue 0x7fdce77ca0a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 57.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fdd94bce040>
├── 'a' --> tensor([[-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398],
│                   [-0.0328, -1.7972, -0.0108],
│                   [ 0.4859, -0.6701, -0.2398]])
└── 'x' --> <FastTreeValue 0x7fdd94bbbe20>
    └── 'c' --> tensor([[ 1.0154,  1.2901,  0.0250,  0.2991],
                        [ 0.8841, -0.1253, -0.4294, -0.0852],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.7 µs ± 54.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]],
       
               [[-0.0328, -1.7972, -0.0108],
                [ 0.4859, -0.6701, -0.2398]]]),
    x: Batch(
           c: tensor([[[ 1.0154,  1.2901,  0.0250,  0.2991],
                       [ 0.8841, -0.1253, -0.4294, -0.0852],
                       [ 0.7056,  0.7251,  1.1095, -0.7267]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398],
               [-0.0328, -1.7972, -0.0108],
               [ 0.4859, -0.6701, -0.2398]]),
    x: Batch(
           c: tensor([[ 1.0154,  1.2901,  0.0250,  0.2991],
                      [ 0.8841, -0.1253, -0.4294, -0.0852],
                      [ 0.7056,  0.7251,  1.1095, -0.7267],
                      [ 1.0154,  1.2901,  0.0250,  0.2991],
                      [ 0.8841, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 2.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
